<a href="https://colab.research.google.com/github/SarathSabu/Python-Notebooks/blob/main/Scheduling_Sports_Equipment_Production.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


## Scheduling Sports Equipment Production


Install amplpy, pandas and other packages.

In [ ]:
!pip install -q amplpy ampltools

Setup AMPL and select solvers.

In [ ]:
# Google Colab & AMPL integration
MODULES, LICENSE_UUID = ["coin", 'gurobi', "cplex", "highs", "gokestrel"], "42fc7eb6-69aa-445d-b655-3ad24d836541"
from amplpy import tools
from ampltools import cloud_platform_name, ampl_notebook, register_magics

# instantiate AMPL object and register magics
if cloud_platform_name() is None:
    ampl = AMPL() # Use local installation of AMPL
else:
    ampl = tools.ampl_notebook(modules=MODULES, license_uuid=LICENSE_UUID, g=globals())

register_magics(ampl_object=ampl)

Licensed to Bundle #6741.7193 expiring 20241231: INFO 645 Prescriptive Analytics, Prof. Paul Brooks, Virginia Commonwealth University.


Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Read data

In [ ]:
import pandas as pd

# Load the Demand Data
demand_data = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/Brendamore_separated_sheets.xlsx',
                            sheet_name="Demand", index_col=0)

# Load the Production Costs Data
production_costs_data = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/Brendamore_separated_sheets.xlsx',
                                      sheet_name="Production Costs", index_col=0)

# Load the Holding Costs Data
holding_costs_data = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/Brendamore_separated_sheets.xlsx',
                                   sheet_name="Holding Costs", index_col=0)

# Define sets
M = list(demand_data.index)
B = list(production_costs_data.columns) # Get the column names from the production_costs_data DataFrame

# Convert demand, production cost, and holding cost data into dictionaries for easy access
demand_ampl = {(i, j): demand_data.loc[j, i] for i in B for j in M}
production_cost_ampl = {(i, j): production_costs_data.loc[j, i] for i in B for j in M}
holding_cost_ampl = {(i, j): holding_costs_data.loc[j, i] for i in B for j in M}

# Define initial inventory, final inventory, production capacity, and inventory capacity
initial_inventory = {B[0]: 7000, B[1]: 5000} # Use B[0] and B[1] to access the correct keys
final_inventory = {B[0]: 3000, B[1]: 3000}   # Use B[0] and B[1] to access the correct keys
production_capacity = 32000
inventory_capacity = 20000

# Print the loaded data to verify
print("Months:", M)
print("Demand (AMPL format):", demand_ampl)
print("Production Cost (AMPL format):", production_cost_ampl)
print("Holding Cost (AMPL format):", holding_cost_ampl)
print("Initial Inventory:", initial_inventory)
print("Final Inventory:", final_inventory)

Months: [1, 2, 3, 4, 5, 6]
Demand (AMPL format): {('footballs', 1): 15000, ('footballs', 2): 25000, ('footballs', 3): 20000, ('footballs', 4): 5000, ('footballs', 5): 2500, ('footballs', 6): 5000, ('soccer balls', 1): 10000, ('soccer balls', 2): 15000, ('soccer balls', 3): 10000, ('soccer balls', 4): 5000, ('soccer balls', 5): 5000, ('soccer balls', 6): 7500}
Production Cost (AMPL format): {('footballs', 1): 13.8, ('footballs', 2): 13.9, ('footballs', 3): 12.95, ('footballs', 4): 12.6, ('footballs', 5): 12.55, ('footballs', 6): 12.7, ('soccer balls', 1): 10.85, ('soccer balls', 2): 10.55, ('soccer balls', 3): 10.5, ('soccer balls', 4): 10.5, ('soccer balls', 5): 10.55, ('soccer balls', 6): 10.0}
Holding Cost (AMPL format): {('footballs', 1): 0.6900000000000001, ('footballs', 2): 0.6950000000000001, ('footballs', 3): 0.6475, ('footballs', 4): 0.63, ('footballs', 5): 0.6275000000000001, ('footballs', 6): 0.635, ('soccer balls', 1): 0.5425, ('soccer balls', 2): 0.5275000000000001, ('socce


Define model.

In [ ]:
ampl.eval('''

reset;

# Define sets for months and products
set M;          # Months {1, 2, 3, 4, 5, 6}
set B;          # Products {footballs, soccer balls}
set Mplus0;  # Add month 0 to handle initial inventory

# Parameters
param Demand {i in B, j in M};           # Demand for product i in month j
param Production_cost {i in B, j in M};  # Production cost of product i in month j
param Holding_cost {i in B, j in M};     # Holding cost of product i in month j
param initial_inventory {i in B};   # Initial inventory for product i
param final_inventory {i in B};     # Final required inventory for product i at the end of month 6
param production_capacity;     # Maximum total production in a month (32,000)
param inventory_capacity;      # Maximum total inventory at the end of the month (20,000)

# Decision Variables
var x {i in B, j in M} >= 0;             # Number of product i to produce in month j
var z {i in B, j in Mplus0} >= 0;        # Inventory of product i at the end of month j

# Objective: Minimize total cost (production cost + holding cost)
minimize Total_Cost:
    sum {i in B, j in M}
        (Production_cost[i,j] * x[i,j] + Holding_cost[i,j] * z[i,j]);

# Constraints

# Inventory balance: Inventory at the end of the previous month + production must meet demand
subject to Inventory_Balance {i in B, j in M}:
    z[i,j-1] + x[i,j] - Demand[i,j] = z[i,j];

# Production capacity: At most 32,000 units (footballs + soccer balls) can be produced in a month
subject to Production_Capacity {j in M}:
    sum {i in B} x[i,j] <= production_capacity;

# Inventory capacity: At most 20,000 units can be stored at the end of any month
subject to Inventory_Capacity {j in M}:
    sum {i in B} z[i,j] <= inventory_capacity;

# Initial inventory constraints
subject to Initial_Inventory {i in B}:
    z[i,0] = initial_inventory[i];

# Final inventory constraints (end of month 6)
subject to Final_Inventory {i in B}:
    z[i,6] = final_inventory[i];

''')


Provide data to the model.

In [ ]:
# Assign sets to AMPL
ampl.set['M'] = M
ampl.set['B'] = B
Mplus0 = [0] + M  # Define Mplus0 by adding 0 to M
ampl.set['Mplus0'] = Mplus0

# Assign parameters to AMPL
ampl.param['Demand'] = demand_ampl
ampl.param['Production_cost'] = production_cost_ampl
ampl.param['Holding_cost'] = holding_cost_ampl
ampl.param['initial_inventory'] = initial_inventory
ampl.param['final_inventory'] = final_inventory
ampl.param['production_capacity'] = production_capacity
ampl.param['inventory_capacity'] = inventory_capacity


Display problem formulation.

In [ ]:
ampl.eval('''expand;''')
# ampl.eval('''expand x;''')
# ampl.eval('''expand z;''')

minimize Total_Cost:
	13.8*x['footballs',1] + 13.9*x['footballs',2] + 12.95*x['footballs',3]
	 + 12.6*x['footballs',4] + 12.55*x['footballs',5] + 
	12.7*x['footballs',6] + 10.85*x['soccer balls',1] + 
	10.55*x['soccer balls',2] + 10.5*x['soccer balls',3] + 
	10.5*x['soccer balls',4] + 10.55*x['soccer balls',5] + 
	10*x['soccer balls',6] + 0.69*z['footballs',1] + 0.695*z['footballs',2]
	 + 0.6475*z['footballs',3] + 0.63*z['footballs',4] + 
	0.6275*z['footballs',5] + 0.635*z['footballs',6] + 
	0.5425*z['soccer balls',1] + 0.5275*z['soccer balls',2] + 
	0.525*z['soccer balls',3] + 0.525*z['soccer balls',4] + 
	0.5275*z['soccer balls',5] + 0.5*z['soccer balls',6];

subject to Inventory_Balance['footballs',1]:
	x['footballs',1] + z['footballs',0] - z['footballs',1] = 15000;

subject to Inventory_Balance['footballs',2]:
	x['footballs',2] + z['footballs',1] - z['footballs',2] = 25000;

subject to Inventory_Balance['footballs',3]:
	x['footballs',3] + z['footballs',2] - z['footballs',3] = 20000

Set solver and solve.

In [ ]:
ampl.setOption('solver', 'cplex')
ampl.solve()

CPLEX 22.1.1:  - Version identifier: 22.1.1.0 | 2022-11-28 | 9160aff4d
 - CPXPARAM_Simplex_Display                         0
 - CPXPARAM_MIP_Display                             0
 - CPXPARAM_Barrier_Display                         0
CPLEX 22.1.1: optimal solution; objective 1448750
12 simplex iterations


Print solution and results.

In [ ]:
# Print the total minimized cost
obj = ampl.getObjective('Total_Cost')
print("\n")
print("Total cost is: ", obj.get().value(), "\n")

# Print the units produced for each product in each month
print("Units Produced:")
ampl.display('x')

# Print the inventory levels for each product at the end of each month
print("\nInventory Levels at the end of each month:")
ampl.display('z')




Total cost is:  1448750.0 

Units Produced:
x :=
footballs      1   16000
footballs      2   17000
footballs      3   20000
footballs      4    5000
footballs      5    2500
footballs      6    8000
'soccer balls' 1    5000
'soccer balls' 2   15000
'soccer balls' 3   10000
'soccer balls' 4    5000
'soccer balls' 5    5000
'soccer balls' 6   10500
;


Inventory Levels at the end of each month:
z :=
footballs      0   7000
footballs      1   8000
footballs      2      0
footballs      3      0
footballs      4      0
footballs      5      0
footballs      6   3000
'soccer balls' 0   5000
'soccer balls' 1      0
'soccer balls' 2      0
'soccer balls' 3      0
'soccer balls' 4      0
'soccer balls' 5      0
'soccer balls' 6   3000
;



The optimal solution is to minimize the total cost to 1,448,750(in dollars) over the six month planning period. Total production capacity is 1,440,825(in dollars) and a holding cost of 8,940(in dollars). The total production of each type of ball is as given above. The company managed to meet its monthly demand adhering to its production capacity and maintaining inventory levels.